In [1]:
from pathlib import Path
import sys, importlib, warnings
warnings.filterwarnings("ignore")

# locate repo root
cwd = Path().resolve()
root = None
for p in [cwd, *cwd.parents]:
    if (p/"src").is_dir():
        root = p; break
assert root
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

import pandas as pd
import numpy as np

import src.config, src.backtest.backtester, src.utils.plotting
importlib.reload(src.config); importlib.reload(src.backtest.backtester); importlib.reload(src.utils.plotting)

from src.config import Settings
from src.backtest.backtester import Backtester, BacktestConfig
from src.utils.plotting import Plotter

cfg = Settings()
cfg




Settings(start='2015-07-01', end='2025-07-31', tickers=['TSLA', 'BND', 'SPY'], risk_free_rate=0.02, seed=42, data_raw_dir=WindowsPath('../data/raw'), data_processed_dir=WindowsPath('../data/processed'), reports_figures_dir=WindowsPath('../reports/figures'))

In [2]:
# Load processed features (Task 1)
features = pd.read_csv(cfg.data_processed_dir / "merged_features.csv", parse_dates=["Date"]).set_index("Date")
rets = features[["TSLA_ret","BND_ret","SPY_ret"]].rename(columns={"TSLA_ret":"TSLA","BND_ret":"BND","SPY_ret":"SPY"}).dropna()

# Choose strategy weights from Task 4 (pick Max Sharpe by default)
weights_path = cfg.reports_figures_dir.parent / "interim" / "weights_max_sharpe.csv"
if not weights_path.exists():
    # fallback to Min Vol if Max Sharpe not generated
    weights_path = cfg.reports_figures_dir.parent / "interim" / "weights_min_vol.csv"

w = pd.read_csv(weights_path, header=None, index_col=0).squeeze()
strategy_weights = {k: float(v) for k, v in w.items() if k in ["TSLA","BND","SPY"]}

strategy_weights


{'TSLA': 0.0, 'BND': 0.0, 'SPY': 1.0}

In [3]:
cfg_bt = BacktestConfig(
    start="2024-08-01",
    end="2025-07-31",
    rebalance="none",      # or "monthly"
    rf_annual=cfg.risk_free_rate
)

bt = Backtester(rets, cfg_bt)
res = bt.run(strategy_weights=strategy_weights, benchmark_weights={"SPY":0.60,"BND":0.40})

display(res.stats)
res.cumrets.tail()


,annual_ret,annual_vol,sharpe
strategy,0.209436,0.202015,0.93773
benchmark,0.131067,0.123108,0.90219


,strategy,benchmark
Date,,
2025-07-24,1.182891,1.119600
2025-07-25,1.187888,1.123600
2025-07-28,1.187590,1.122740
2025-07-29,1.184457,1.123036
2025-07-30,1.182966,1.121066


In [4]:
pl = Plotter(cfg.reports_figures_dir)

fig_path = pl.cumulative_returns(
    cum_df=res.cumrets,
    title="Backtest — Strategy vs 60/40 SPY/BND (Aug 2024 to Jul 2025)",
    fname="backtest_cumreturns.png"
)

out_dir = cfg.reports_figures_dir.parent / "interim"
out_dir.mkdir(parents=True, exist_ok=True)

res.daily.to_csv(out_dir / "backtest_daily_returns.csv")
res.cumrets.to_csv(out_dir / "backtest_cumulative_curves.csv")
res.stats.to_csv(out_dir / "backtest_stats.csv")

fig_path, out_dir / "backtest_stats.csv"


(WindowsPath('../reports/figures/backtest_cumreturns.png'),
 WindowsPath('../reports/interim/backtest_stats.csv'))

In [5]:
stats = res.stats.copy()
msg = (
    f"Strategy:   μ={stats.loc['strategy','annual_ret']:.3%}, "
    f"σ={stats.loc['strategy','annual_vol']:.3%}, "
    f"Sharpe={stats.loc['strategy','sharpe']:.2f}\n"
    f"Benchmark:  μ={stats.loc['benchmark','annual_ret']:.3%}, "
    f"σ={stats.loc['benchmark','annual_vol']:.3%}, "
    f"Sharpe={stats.loc['benchmark','sharpe']:.2f}\n"
    f"Conclusion: {'Outperformed' if stats.loc['strategy','sharpe'] > stats.loc['benchmark','sharpe'] else 'Underperformed'} the benchmark on Sharpe."
)
print(msg)


Strategy:   μ=20.944%, σ=20.202%, Sharpe=0.94
Benchmark:  μ=13.107%, σ=12.311%, Sharpe=0.90
Conclusion: Outperformed the benchmark on Sharpe.
